In [ ]:
!wget -nc https://s3.amazonaws.com/content.udacity-data.com/courses/nd188/flower_data.zip
!unzip -qq flower_data.zip

--2020-08-19 14:24:49--  https://s3.amazonaws.com/content.udacity-data.com/courses/nd188/flower_data.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.137.230
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.137.230|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 311442766 (297M) [application/zip]
Saving to: ‘flower_data.zip’

flower_data.zip     100%[===================>] 297.01M  16.1MB/s    in 20s     

2020-08-19 14:25:10 (14.8 MB/s) - ‘flower_data.zip’ saved [311442766/311442766]



In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data

import torchvision.transforms as transforms
import torchvision.datasets as datasets

from sklearn import decomposition
from sklearn import manifold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import numpy as np

import copy
import random
import time


In [ ]:
from torch.utils.data import Dataset, DataLoader

In [ ]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [ ]:
train_transforms = transforms.Compose([transforms.Resize(224),
                                       transforms.CenterCrop(224),
                                       transforms.RandomRotation(30),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.RandomVerticalFlip(),
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])


In [ ]:
train_data = datasets.ImageFolder(root = '/content/flower_data/train',transform=train_transforms)

In [ ]:
VALID_RATIO = 0.25

n_train_examples = int(len(train_data) * VALID_RATIO)
n_valid_examples = len(train_data) - n_train_examples
print(n_train_examples)
print(n_valid_examples)

train_data, valid_data = data.random_split(train_data, 
                                           [n_train_examples, n_valid_examples])

1638
4914


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
import copy
import random
import time
import torchvision.models as models

In [ ]:
model = models.resnet101(pretrained=True).to(device)

for param in model.parameters():
  param.requires_grad=True
    
model.fc = nn.Sequential(
               nn.Linear(2048, 496),
               nn.ReLU(inplace=True),
               nn.Linear(496, 102)).to(device)

Downloading: "https://download.pytorch.org/models/resnet101-5d3b4d8f.pth" to /root/.cache/torch/hub/checkpoints/resnet101-5d3b4d8f.pth


In [ ]:
BATCH_SIZE = 16
train_iterator = data.DataLoader(train_data, 
                                 shuffle = True, 
                                 batch_size = BATCH_SIZE)

valid_iterator = data.DataLoader(valid_data, 
                                 batch_size = BATCH_SIZE)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 43,567,158 trainable parameters


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.005, momentum=0.9, dampening=0, weight_decay=0.0001, nesterov=True)

criterion = criterion.to(device)

In [ ]:
def calculate_accuracy(y_pred, y):
    top_pred = y_pred.argmax(1, keepdim = True)
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    acc = correct.float() / y.shape[0]
    return acc

In [ ]:
def train(model, iterator, optimizer, criterion, device):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for (x, y) in iterator:
        
        x = x.to(device)
        y = y.to(device)
        y = y.type(torch.LongTensor)
        y = y.to(device)
        
        optimizer.zero_grad()
                
        y_pred = model(x)
        
        loss = criterion(y_pred, y)
        
        acc = calculate_accuracy(y_pred, y)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion, device):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
        
        for (x, y) in iterator:

            x = x.to(device)
            y = y.to(device)
            y = y.type(torch.LongTensor)
            y = y.to(device)

            y_pred = model(x)

            loss = criterion(y_pred, y)

            acc = calculate_accuracy(y_pred, y)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
EPOCHS = 200

best_valid_loss = float('inf')

for epoch in range(EPOCHS):
    
    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion, device)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion, device)
    
    if valid_loss < best_valid_loss :
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 3m 14s
	Train Loss: 4.054 | Train Acc: 12.72%
	 Val. Loss: 2.937 |  Val. Acc: 32.12%
Epoch: 02 | Epoch Time: 3m 15s
	Train Loss: 2.271 | Train Acc: 45.53%
	 Val. Loss: 1.589 |  Val. Acc: 59.48%
Epoch: 03 | Epoch Time: 3m 15s
	Train Loss: 1.181 | Train Acc: 68.63%
	 Val. Loss: 1.117 |  Val. Acc: 69.05%
Epoch: 04 | Epoch Time: 3m 14s
	Train Loss: 0.783 | Train Acc: 78.07%
	 Val. Loss: 1.030 |  Val. Acc: 71.33%
Epoch: 05 | Epoch Time: 3m 14s
	Train Loss: 0.620 | Train Acc: 82.75%
	 Val. Loss: 1.039 |  Val. Acc: 73.68%
Epoch: 06 | Epoch Time: 3m 14s
	Train Loss: 0.458 | Train Acc: 87.22%
	 Val. Loss: 0.928 |  Val. Acc: 75.12%
Epoch: 07 | Epoch Time: 3m 15s
	Train Loss: 0.398 | Train Acc: 88.25%
	 Val. Loss: 0.576 |  Val. Acc: 84.66%
Epoch: 08 | Epoch Time: 3m 16s
	Train Loss: 0.299 | Train Acc: 92.25%
	 Val. Loss: 0.911 |  Val. Acc: 78.47%
Epoch: 09 | Epoch Time: 3m 17s
	Train Loss: 0.243 | Train Acc: 93.20%
	 Val. Loss: 0.712 |  Val. Acc: 81.53%
Epoch: 10 | Epoch T

KeyboardInterrupt: ignored